In [10]:
import ee
import geemap

ee.Initialize()


# Define the Amazon Basin boundary
amazon_region = ee.Geometry.Polygon(
    [[[-80.0, 10.0], [-20.0, 10.0], [-20.0, -50.0], [-80.0, -50.0], [-80.0, 10.0]]]
)

# Load datasets

gedi = ee.FeatureCollection('LARSE/GEDI/GEDI04_A_002_INDEX')\
        .filter('time_start > "2023-01-01" && time_end < "2023-12-01"')\
        .filterBounds(amazon_region)
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(amazon_region)
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(amazon_region)
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30') \
          .filterBounds(amazon_region) \
          .mosaic()

# Preprocessing GEDI data
gedi_filtered = (
    gedi.filter(ee.Filter.eq('l4_quality_flag', 1))
    .filter(ee.Filter.lt('agbd_se', 50))
)

# Filter terrain slope > 30 degrees
slope = ee.Terrain.slope(dem)
gedi_filtered = gedi_filtered.filter(ee.Filter.lt('slope', 30))

# Mask land cover categories unsuitable for AGB analysis
world_cover = ee.Image('ESA/WorldCover/v100/2020')
unsuitable_land_cover = [50, 60, 70, 80, 90, 100]
land_cover_mask = world_cover.remap(
    unsuitable_land_cover, [0] * len(unsuitable_land_cover), 1
)

gedi_filtered = gedi_filtered.filterBounds(
    world_cover.updateMask(land_cover_mask)
)

# Sentinel-1 preprocessing
def mask_sentinel1(image):
    edge = image.lt(-30.0)
    return image.updateMask(edge.Not())

sentinel1 = sentinel1.map(mask_sentinel1)

# Sentinel-2 preprocessing
# def mask_clouds(image):
#     qa = image.select('QA60')
#     cloud_mask = qa.bitwiseAnd(int(1 << 10)).eq(0).And(qa.bitwiseAnd(int(1 << 11)).eq(0))
#     return image.updateMask(cloud_mask)

# Define the bitmasks
cloud_bit_mask = ee.Number(1 << 5)  # Cloud bit is in the 6th bit position
cirrus_bit_mask = ee.Number(1 << 9)  # Cirrus bit is in the 10th bit position


# Apply the mask using bitwise AND to check that both cloud and cirrus bits are 0
def mask_clouds(image):
    qa = image.select('QA60')  # Select the QA60 band that holds cloud and cirrus bit information
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)

sentinel2 = sentinel2.map(mask_clouds).select(['B4', 'B3', 'B2', 'B8', 'B11', 'B12'])

# Compute NDVI and other indices
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2'),
        },
    ).rename('EVI')
    return image.addBands([ndvi, evi])

sentinel2 = sentinel2.map(add_indices)

# Combine all features
feature_stack = sentinel2.median().addBands(sentinel1.median()).addBands(dem)

# Sample the remote sensing data at GEDI footprint locations
training_data = feature_stack.sampleRegions(
    collection=gedi_filtered, properties=['agbd'], scale=100, tileScale=5
)

# Train a Gradient Tree Boost model
trained_model = ee.Classifier.smileGradientTreeBoost(
    numberOfTrees=100, 
    samplingRate=0.1, 
    maxNodes=10
).train(features=training_data, classProperty='agbd', inputProperties=feature_stack.bandNames())

# Predict AGB
agb_prediction = feature_stack.classify(trained_model)

# Post-processing
agb_prediction = agb_prediction.updateMask(land_cover_mask)

# Visualization
agb_vis_params = {'min': 0, 'max': 100, 'palette': ['blue', 'green', 'yellow', 'red']}
Map = geemap.Map()
Map.centerObject(amazon_region, 6)
Map.addLayer(agb_prediction, agb_vis_params, 'AGB Prediction')
Map


EEException: Feature, argument 'geometry': Invalid type.
Expected type: Geometry.
Actual type: Image<[Map]>.